# 1. Initialize

### Libraries and Spark context

In [2]:
import pyspark
from pyspark.sql import SparkSession
from probetrip.trip import Trip

In [12]:
'''
.config("spark.dynamicAllocation.enabled", "true") \
.config("spark.dynamicAllocation.minExecutors", "1").config("spark.dynamicAllocation.maxExecutors", "3") \
'''
spark = SparkSession.builder.master("yarn").appName("3_create_trip") \
        .config("spark.executor.cores", "2").config("spark.driver.cores", "1") \
        .config("spark.executor.memory", "6g").config("spark.driver.memory", "2g") \
        .config("spark.executor.instances", "3") \
        .config("spark.sql.catalogImplementation", "hive") \
        .config("metastore.catalog.default", "hive") \
        .enableHiveSupport() \
        .getOrCreate()

24/07/19 09:03:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/07/19 09:03:35 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [127]:
spark.stop()

### Required parameter

**Probe table structure** (input table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example</th>
    </tr>
    <tr>
        <td style='text-align: left;'>vehicle_id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>vHscKFQJlnlim4T7dk/9c7xACjU</td>
    </tr>
    <tr>
        <td style='text-align: left;'>gpsvalid</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>lat</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>13.80699</td>
    </tr>
    <tr>
        <td style='text-align: left;'>lon</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>100.54299</td>
    </tr>
    <tr>
        <td style='text-align: left;'>date_time</td>
        <td style='text-align: left;'>timestamp</td>
        <td style='text-align: left;'>2023-02-01 12:34:37</td>
    </tr>
    <tr>
        <td style='text-align: left;'>speed</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0</td>
    </tr>
    <tr>
        <td style='text-align: left;'>heading</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>for_hire_light</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>engine_acc</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>0</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202302</td>
    </tr>
</table>

**Trip table structure** (output table)
<table style='float:left;'>
    <tr>
        <th style='text-align: left;'>Name</th>
        <th style='text-align: left;'>Data type</th>
        <th style='text-align: left;'>Example (Stay)</th>
        <th style='text-align: left;'>Example (Move)</th>
    </tr>
    <tr>
        <td style='text-align: left;'>id</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
        <td style='text-align: left;'>+1indEOKr/ikPVrJQTVjw4FGxBE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_date</td>
        <td style='text-align: left;'>date </td>
        <td style='text-align: left;'>2023-01-01</td>
        <td style='text-align: left;'>2023-01-01</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>trip_sequence</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>1</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>mobility_type</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>STAY</td>
        <td style='text-align: left;'>MOVE</td>
    </tr>
    <tr>
        <td style='text-align: left;'>passenger</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>0</td>
        <td style='text-align: left;'>1</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_distance (km)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>209.57</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_time (minutes)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>390.3</td>
    </tr>
    <tr>
        <td style='text-align: left;'>overall_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>318.53</td>
        <td style='text-align: left;'>32.22</td>
    </tr>
    <tr>
        <td style='text-align: left;'>average_speed (km/h)</td>
        <td style='text-align: left;'>float</td>
        <td style='text-align: left;'>0.0</td>
        <td style='text-align: left;'>28.8</td>
    </tr>
    <tr>
        <td style='text-align: left;'>start_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>00:02:49</td>
        <td style='text-align: left;'>05:21:21</td>
    </tr>
    <tr>
        <td style='text-align: left;'>end_time</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>05:21:21</td>
        <td style='text-align: left;'>11:51:39</td>
    </tr>
    <tr>
        <td style='text-align: left;'>total_points</td>
        <td style='text-align: left;'>integer</td>
        <td style='text-align: left;'>121</td>
        <td style='text-align: left;'>283</td>
    </tr>
    <tr>
        <td style='text-align: left;'>sub_district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Lam Phak Kut</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>district</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Thanyaburi</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>province</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>Pathum Thani</td>
        <td style='text-align: left;'></td>
    </tr>
    <tr>
        <td style='text-align: left;'>point_list</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>[[14.032970,100.786000]]</td>
        <td style='text-align: left;'>[["05:21:21",14.03297,100.786],["05:22:21",14.0346,100.78606], ..., ["11:51:39",14.03296,100.78601]]</td>
    </tr>
    <tr>
        <td style='text-align: left;'>partition_month</td>
        <td style='text-align: left;'>string</td>
        <td style='text-align: left;'>202301</td>
        <td style='text-align: left;'>202301</td>
    </tr>
</table>

In [ ]:
database_name = "probe_taxi"
probe_table = "probe_data"
trip_table = "trip_point"
spark_stat_table='spark_stat'
shape_path = "dataset/tha_shapefiles/tha_admbnda_adm3_rtsd_20220121.shp"

start_date = '20230101'
end_date = '20230101'
distance_threshold = 0.15 # kilometers
duration_threshold = 8    # minutes

# 2. Execution

In [6]:
trip = Trip(spark, database_name, probe_table, trip_table, shape_path)

In [8]:
trip.generate_trip(start_date, end_date, distance_threshold, duration_threshold, verbose=False, spark_stat_table=spark_stat_table)

Step 6: 100%|██████████| 6/6 [55:47<00:00, 557.85s/it]                          


Generate Probe trip point by user on 20230101 to 20230131 successfuly! 	 | Process time: 55.79 mins
